In [1]:
from pyspark.sql import functions as F
from pyspark.sql.window import Window
import pandas as pd

In [2]:
df = spark.createDataFrame(
[('11','1','1152397078','VVVVM',1,'3/5/2020',1,5),
('11','1','1152944770','VVVVV',1,'3/6/2020',2,5),
('11','1','1153856408','VVVVV',1,'3/15/2020',3,5),
('11','2','1155884040','MVVVV',1,'4/2/2020',4,5),
('11','2','1156854301','MMVVV',0,'4/17/2020',5,5),
('12','1','1156854302','VVVVM',1,'3/6/2020',1,3),
('12','1','1156854303','VVVVV',1,'3/7/2020',2,3),
('12','2','1156854304','MVVVV',1,'3/16/2020',3,3)
]
,["consumer_id","product_id","TRX_ID","pattern","loyal","trx_date","row_num","mx"])
df.show()

+-----------+----------+----------+-------+-----+---------+-------+---+
consumer_id|product_id| TRX_ID|pattern|loyal| trx_date|row_num| mx|
+-----------+----------+----------+-------+-----+---------+-------+---+
 11| 1|1152397078| VVVVM| 1| 3/5/2020| 1| 5|
 11| 1|1152944770| VVVVV| 1| 3/6/2020| 2| 5|
 11| 1|1153856408| VVVVV| 1|3/15/2020| 3| 5|
 11| 2|1155884040| MVVVV| 1| 4/2/2020| 4| 5|
 11| 2|1156854301| MMVVV| 0|4/17/2020| 5| 5|
 12| 1|1156854302| VVVVM| 1| 3/6/2020| 1| 3|
 12| 1|1156854303| VVVVV| 1| 3/7/2020| 2| 3|
 12| 2|1156854304| MVVVV| 1|3/16/2020| 3| 3|
+-----------+----------+----------+-------+-----+---------+-------+---+

In [3]:
from pypsark.sql import functions as F
from pyspark.sql.window import Window

w = Window().partitionBy("consumer_id").orderBy('row_num')
lead=F.lead("loyal").over(w)
df.withColumn("Flag", F.when(((F.col("loyal")==1)\
                             &((lead==0)|(lead.isNull()))),F.lit(1))\
                       .otherwise(F.lit(0))).show()

+-----------+----------+----------+-------+-----+---------+-------+---+----+
consumer_id|product_id| TRX_ID|pattern|loyal| trx_date|row_num| mx|Flag|
+-----------+----------+----------+-------+-----+---------+-------+---+----+
 11| 1|1152397078| VVVVM| 1| 3/5/2020| 1| 5| 0|
 11| 1|1152944770| VVVVV| 1| 3/6/2020| 2| 5| 0|
 11| 1|1153856408| VVVVV| 1|3/15/2020| 3| 5| 0|
 11| 2|1155884040| MVVVV| 1| 4/2/2020| 4| 5| 1|
 11| 2|1156854301| MMVVV| 0|4/17/2020| 5| 5| 0|
 12| 1|1156854302| VVVVM| 1| 3/6/2020| 1| 3| 0|
 12| 1|1156854303| VVVVV| 1| 3/7/2020| 2| 3| 0|
 12| 2|1156854304| MVVVV| 1|3/16/2020| 3| 3| 1|
+-----------+----------+----------+-------+-----+---------+-------+---+----+

In [4]:
list=[[1,23,90],
     [2,12,45],
     [2,38,80],
     [2,91,62],
     [1,11,21],
     [1,29,57],
     [1,13,68],
     [2,14,19]]

df=spark.createDataFrame(list,['id','column_b','column_a'])


df.select("id","column_a","column_b").show()

+---+--------+--------+
 id|column_a|column_b|
+---+--------+--------+
 1| 90| 23|
 2| 45| 12|
 2| 80| 38|
 2| 62| 91|
 1| 21| 11|
 1| 57| 29|
 1| 68| 13|
 2| 19| 14|
+---+--------+--------+

In [5]:
You could only use **`collect_list`** over **`only one`** **`window`** and then use *higher order function* **`aggregate`** to get your desired result **`(sum/sum).`**

    df.show() #sample dataframe
    #+---+--------+--------+
    #| id|column_a|column_b|
    #+---+--------+--------+
    #|  1|      90|      23|
    #|  2|      45|      12|
    #|  2|      80|      38|
    #|  2|      62|      91|
    #|  1|      21|      11|
    #|  1|      57|      29|
    #|  1|      68|      13|
    #|  2|      19|      14|
    #+---+--------+--------+
    
    from pyspark.sql import functions as F
    from pyspark.sql.window import Window
    
    
    window=Window().partitionBy("id")
    df.withColumn("column_c",F.collect_list(F.array("column_a","column_b")).over(window))\
      .withColumn("column_c", F.expr("""aggregate(column_c,0,(acc,x)-> int(x[0])+acc)/\
                                   aggregate(column_c,0,(acc,x)-> int(x[1])+acc)""")).show()

    #+---+--------+--------+------------------+
    #| id|column_b|column_a|          column_c|
    #+---+--------+--------+------------------+
    #|  1|      23|      90|3.1052631578947367|
    #|  1|      11|      21|3.1052631578947367|
    #|  1|      29|      57|3.1052631578947367|
    #|  1|      13|      68|3.1052631578947367|
    #|  2|      12|      45|1.3290322580645162|
    #|  2|      38|      80|1.3290322580645162|
    #|  2|      91|      62|1.3290322580645162|
    #|  2|      14|      19|1.3290322580645162|
    #+---+--------+--------+------------------+

**`Instead of:`**(2 windows)

    window=Window().partitionBy("id")
    df.withColumn("colum_c",F.sum(F.col("column_a")).over(window)\
                                  /F.sum(F.col("column_b")).over(window)).show()


In [6]:
from pyspark.sql import functions as f

  
list=[[1,23,90,'2019-2-23'],
     [1,12,45,'2019-2-28'],
     [1,38,80,'2019-3-21'],
     [1,91,62,'2019-3-24'],
     [2,11,21,'2019-3-29'],
     [2,29,57,'2019-1-08'],
     [2,13,68,'2019-1-12'],
     [2,14,19,'2019-1-14']]

df=spark.createDataFrame(list,['account_id','column_b','column_a','event_date'])


df=df.select("account_id","column_a","column_b","event_date")

In [7]:
df.show() #sample data

#+----------+--------+--------+----------+
#|account_id|column_a|column_b|event_date|
#+----------+--------+--------+----------+
#|         1|      90|      23| 2019-2-23|
#|         1|      45|      12| 2019-2-28|
#|         1|      80|      38| 2019-3-21|
#|         1|      62|      91| 2019-3-24|
#|         2|      21|      11| 2019-3-29|
#|         2|      57|      29| 2019-1-08|
#|         2|      68|      13| 2019-1-12|
#|         2|      19|      14| 2019-1-14|
#+----------+--------+--------+----------+

from pyspark.sql import functions as F
from pyspark.sql.window import Window

days = lambda i: (i-1) * 86400

window =\
    Window()\
    .partitionBy(f.col("account_id"))\
    .orderBy(f.col("event_date").cast("timestamp").cast("long"))\
    .rangeBetween(-days(7), 0)

df.withColumn("column_c",F.collect_list(F.array("column_a","column_b")).over(window))\
  .withColumn("column_c", F.expr("""aggregate(column_c,0,(acc,x)-> int(x[0])+acc)/\
                               aggregate(column_c,0,(acc,x)-> int(x[1])+acc)""")).show()

#+----------+--------+--------+----------+------------------+
#|account_id|column_b|column_a|event_date|          column_c|
#+----------+--------+--------+----------+------------------+
#|         1|      23|      90| 2019-2-23|3.9130434782608696|
#|         1|      12|      45| 2019-2-28| 3.857142857142857|
#|         1|      38|      80| 2019-3-21|2.1052631578947367|
#|         1|      91|      62| 2019-3-24|1.1007751937984496|
#|         2|      29|      57| 2019-1-08|1.9655172413793103|
#|         2|      13|      68| 2019-1-12|2.9761904761904763|
#|         2|      14|      19| 2019-1-14|2.5714285714285716|
#|         2|      11|      21| 2019-3-29|1.9090909090909092|
#+----------+--------+--------+----------+------------------+

+----------+--------+--------+----------+
account_id|column_a|column_b|event_date|
+----------+--------+--------+----------+
 1| 90| 23| 2019-2-23|
 1| 45| 12| 2019-2-28|
 1| 80| 38| 2019-3-21|
 1| 62| 91| 2019-3-24|
 2| 21| 11| 2019-3-29|
 2| 57| 29| 2019-1-08|
 2| 68| 13| 2019-1-12|
 2| 19| 14| 2019-1-14|
+----------+--------+--------+----------+

+----------+--------+--------+----------+------------------+
account_id|column_a|column_b|event_date| column_c|
+----------+--------+--------+----------+------------------+
 1| 90| 23| 2019-2-23|3.9130434782608696|
 1| 45| 12| 2019-2-28| 3.857142857142857|
 1| 80| 38| 2019-3-21|2.1052631578947367|
 1| 62| 91| 2019-3-24|1.1007751937984496|
 2| 57| 29| 2019-1-08|1.9655172413793103|
 2| 68| 13| 2019-1-12|2.9761904761904763|
 2| 19| 14| 2019-1-14|2.5714285714285716|
 2| 21| 11| 2019-3-29|1.9090909090909092|
+----------+--------+--------+----------+------------------+

In [8]:
from pyspark.sql import functions as f
from pyspark.sql.window import Window

days = lambda i: i * 86400

window =\
    Window()\
    .partitionBy(f.col("account_id"))\
    .orderBy(f.col("event_date").cast("timestamp").cast("long"))\
    .rangeBetween(-days(7), 0)

df.withColumn("column_c",f.collect_list(f.array("column_a","column_b")).over(window))\
  .withColumn("column_c", f.expr("""aggregate(column_c,0,(acc,x)-> int(x[0])+acc)/\
                               aggregate(column_c,0,(acc,x)-> int(x[1])+acc)""")).show()

+----------+--------+--------+----------+------------------+
account_id|column_a|column_b|event_date| column_c|
+----------+--------+--------+----------+------------------+
 1| 90| 23| 2019-2-23|3.9130434782608696|
 1| 45| 12| 2019-2-28| 3.857142857142857|
 1| 80| 38| 2019-3-21|2.1052631578947367|
 1| 62| 91| 2019-3-24|1.1007751937984496|
 2| 57| 29| 2019-1-08|1.9655172413793103|
 2| 68| 13| 2019-1-12|2.9761904761904763|
 2| 19| 14| 2019-1-14|2.5714285714285716|
 2| 21| 11| 2019-3-29|1.9090909090909092|
+----------+--------+--------+----------+------------------+

In [9]:
from pyspark.sql import functions as f
from pyspark.sql.window import Window

days = lambda i: i * 86400

window =\
    Window()\
    .partitionBy(f.col("account_id"))\
    .orderBy(f.col("event_date").cast("timestamp").cast("long"))\
    .rangeBetween(-days(7), 0)

df.withColumn("column_c",f.collect_list(f.struct\
                                        (f.col("column_a").alias("a"),\
                                         f.col("column_b").alias("b")))\
                                                          .over(window))\
  .withColumn("column_c", f.expr("""aggregate(column_c,0,(acc,x)-> int(x.a)+acc)/\
                               aggregate(column_c,0,(acc,x)-> int(x.b)+acc)""")).explain()

== Physical Plan ==
*(2) Project [account_id#7210L, column_a#7212L, column_b#7211L, event_date#7213, (cast(aggregate(column_c#8464, 0, lambdafunction((cast(lambda x#8472.a as int) + lambda acc#8471), lambda acc#8471, lambda x#8472, false), lambdafunction(lambda id#8473, lambda id#8473, false)) as double) / cast(aggregate(column_c#8464, 0, lambdafunction((cast(lambda x#8475.b as int) + lambda acc#8474), lambda acc#8474, lambda x#8475, false), lambdafunction(lambda id#8476, lambda id#8476, false)) as double)) AS column_c#8470]
+- Window [account_id#7210L, column_a#7212L, column_b#7211L, event_date#7213, collect_list(_w1#8483, 0, 0) windowspecdefinition(account_id#7210L, _w0#8482L ASC NULLS FIRST, specifiedwindowframe(RangeFrame, -604800, currentrow$())) AS column_c#8464], [account_id#7210L], [_w0#8482L ASC NULLS FIRST]
 +- Sort [account_id#7210L ASC NULLS FIRST, _w0#8482L ASC NULLS FIRST], false, 0
 +- Exchange hashpartitioning(account_id#7210L, 200), [id=#2052]
 +- *(1) Project [account_id#7210L, column_a#7212L, column_b#7211L, event_date#7213, cast(cast(event_date#7213 as timestamp) as bigint) AS _w0#8482L, named_struct(a, column_a#7212L, b, column_b#7211L) AS _w1#8483]
 +- *(1) Scan ExistingRDD[account_id#7210L,column_b#7211L,column_a#7212L,event_date#7213]

In [10]:
days = lambda i: i * 86400

window =\
    Window()\
    .partitionBy(f.col("account_id"))\
    .orderBy(f.col("event_date").cast("timestamp").cast("long"))\
    .rangeBetween(-days(7), 0)

df.withColumn("colum_c",F.sum(F.col("column_a")).over(window)\
                              /F.sum(F.col("column_b")).over(window)).explain()


== Physical Plan ==
Window [account_id#4848L, column_b#4849L, column_a#4850L, event_date#4851, (cast(sum(column_a#4850L) windowspecdefinition(account_id#4848L, _w0#6804L ASC NULLS FIRST, specifiedwindowframe(RangeFrame, -604800, currentrow$())) as double) / cast(sum(column_b#4849L) windowspecdefinition(account_id#4848L, _w0#6804L ASC NULLS FIRST, specifiedwindowframe(RangeFrame, -604800, currentrow$())) as double)) AS colum_c#6798], [account_id#4848L], [_w0#6804L ASC NULLS FIRST]
+- Sort [account_id#4848L ASC NULLS FIRST, _w0#6804L ASC NULLS FIRST], false, 0
 +- Exchange hashpartitioning(account_id#4848L, 200), [id=#1453]
 +- *(1) Project [account_id#4848L, column_b#4849L, column_a#4850L, event_date#4851, cast(cast(event_date#4851 as timestamp) as bigint) AS _w0#6804L]
 +- *(1) Scan ExistingRDD[account_id#4848L,column_b#4849L,column_a#4850L,event_date#4851]

In [11]:
== Physical Plan ==
Window [id#16L, column_b#17L, column_a#18L, (cast(sum(column_a#18L) windowspecdefinition(id#16L, specifiedwindowframe(RowFrame, unboundedpreceding$(), unboundedfollowing$())) as double) / cast(sum(column_b#17L) windowspecdefinition(id#16L, specifiedwindowframe(RowFrame, unboundedpreceding$(), unboundedfollowing$())) as double)) AS colum_c#2635], [id#16L]
+- Sort [id#16L ASC NULLS FIRST], false, 0
   +- Exchange hashpartitioning(id#16L, 200), [id=#722]
      +- *(1) Scan ExistingRDD[id#16L,column_b#17L,column_a#18L]

+---+--------+--------+------------------+
 id|column_b|column_a| column_c|
+---+--------+--------+------------------+
 1| 23| 90|3.1052631578947367|
 1| 11| 21|3.1052631578947367|
 1| 29| 57|3.1052631578947367|
 1| 13| 68|3.1052631578947367|
 2| 12| 45|1.3290322580645162|
 2| 38| 80|1.3290322580645162|
 2| 91| 62|1.3290322580645162|
 2| 14| 19|1.3290322580645162|
+---+--------+--------+------------------+

In [13]:
df.withColumn("colum_c",F.sum(F.col("column_a")).over(window)\
                              /F.sum(F.col("column_b")).over(window)).explain()

== Physical Plan ==
Window [account_id#4848L, column_b#4849L, column_a#4850L, event_date#4851, (cast(sum(column_a#4850L) windowspecdefinition(account_id#4848L, _w0#6818L ASC NULLS FIRST, specifiedwindowframe(RangeFrame, -604800, currentrow$())) as double) / cast(sum(column_b#4849L) windowspecdefinition(account_id#4848L, _w0#6818L ASC NULLS FIRST, specifiedwindowframe(RangeFrame, -604800, currentrow$())) as double)) AS colum_c#6812], [account_id#4848L], [_w0#6818L ASC NULLS FIRST]
+- Sort [account_id#4848L ASC NULLS FIRST, _w0#6818L ASC NULLS FIRST], false, 0
 +- Exchange hashpartitioning(account_id#4848L, 200), [id=#1492]
 +- *(1) Project [account_id#4848L, column_b#4849L, column_a#4850L, event_date#4851, cast(cast(event_date#4851 as timestamp) as bigint) AS _w0#6818L]
 +- *(1) Scan ExistingRDD[account_id#4848L,column_b#4849L,column_a#4850L,event_date#4851]

In [14]:
== Physical Plan ==
Window [account_id#4848L, column_b#4849L, column_a#4850L, event_date#4851, (cast(sum(column_a#4850L) windowspecdefinition(account_id#4848L, _w0#6818L ASC NULLS FIRST, specifiedwindowframe(RangeFrame, -604800, currentrow$())) as double) / cast(sum(column_b#4849L) windowspecdefinition(account_id#4848L, _w0#6818L ASC NULLS FIRST, specifiedwindowframe(RangeFrame, -604800, currentrow$())) as double)) AS colum_c#6812], [account_id#4848L], [_w0#6818L ASC NULLS FIRST]
+- Sort [account_id#4848L ASC NULLS FIRST, _w0#6818L ASC NULLS FIRST], false, 0
   +- Exchange hashpartitioning(account_id#4848L, 200), [id=#1492]
      +- *(1) Project [account_id#4848L, column_b#4849L, column_a#4850L, event_date#4851, cast(cast(event_date#4851 as timestamp) as bigint) AS _w0#6818L]
         +- *(1) Scan ExistingRDD[account_id#4848L,column_b#4849L,column_a#4850L,event_date#4851]

In [15]:
window=Window().partitionBy("id")
df.withColumn("colum_c",F.sum(F.col("column_a")/F.col("column_b")).over(window)).show()

+---+--------+--------+-----------------+
 id|column_b|column_a| colum_c|
+---+--------+--------+-----------------+
 1| 23| 90|13.01842085950032|
 1| 11| 21|13.01842085950032|
 1| 29| 57|13.01842085950032|
 1| 13| 68|13.01842085950032|
 2| 12| 45|7.893724696356275|
 2| 38| 80|7.893724696356275|
 2| 91| 62|7.893724696356275|
 2| 14| 19|7.893724696356275|
+---+--------+--------+-----------------+

In [16]:
window=Window().partitionBy("id")
df.withColumn("colum_c",F.sum(F.col("column_a")).over(window)/F.sum(F.col("column_b")).over(window)).show()

== Physical Plan ==
Window [id#16L, column_b#17L, column_a#18L, (cast(sum(column_a#18L) windowspecdefinition(id#16L, specifiedwindowframe(RowFrame, unboundedpreceding$(), unboundedfollowing$())) as double) / cast(sum(column_b#17L) windowspecdefinition(id#16L, specifiedwindowframe(RowFrame, unboundedpreceding$(), unboundedfollowing$())) as double)) AS colum_c#2635], [id#16L]
+- Sort [id#16L ASC NULLS FIRST], false, 0
 +- Exchange hashpartitioning(id#16L, 200), [id=#722]
 +- *(1) Scan ExistingRDD[id#16L,column_b#17L,column_a#18L]

In [17]:
from pyspark.sql import functions as F

In [18]:
list1=[['abcde00000qMQ00001',['casa', 'alejado', 'buen', 'gusto']],
       ['abcde00000qMq00002',['clientes', 'contentos', 'servi']],
       ['abcde00000qMQ00003',['resto', 'bien']               ]]

df1=spark.createDataFrame(list1,['ID','MeaningfulWords'])

df1.show()

+------------------+--------------------+
 ID| MeaningfulWords|
+------------------+--------------------+
abcde00000qMQ00001|[casa, alejado, b...|
abcde00000qMq00002|[clientes, conten...|
abcde00000qMQ00003| [resto, bien]|
+------------------+--------------------+

In [19]:


list2=[[ 1.68,     'casa'],
       [  2.8,  'alejado'],
       [ 1.03,     'buen'],
       [ 3.68,    'gusto'],
       [ 0.68, 'clientes'],
       [  2.1,'contentos'],
       [ 2.68,    'servi'],
       [ 1.18,    'resto'],
       [ 1.98,    'bien']]

df2=spark.createDataFrame(list2,['score','word'])

df2.show()

+-----+---------+
score| word|
+-----+---------+
 1.68| casa|
 2.8| alejado|
 1.03| buen|
 3.68| gusto|
 0.68| clientes|
 2.1|contentos|
 2.68| servi|
 1.18| resto|
 1.98| bien|
+-----+---------+

In [20]:
from pyspark.sql import functions as F
df1.join(df2, F.expr("""array_contains(MeaningfulWords,word)"""))\
   .groupBy("ID").agg(F.first("MeaningfulWords").alias("MeaningfullWords")\
                      ,F.collect_list("score").alias("ScoreList")\
                      ,F.mean("score").alias("MeanScore"))\
                      .show(truncate=False)

+------------------+----------------------------+-----------------------+------------------+
ID |MeaningfullWords |ScoreList |MeanScore |
+------------------+----------------------------+-----------------------+------------------+
abcde00000qMQ00003|[resto, bien] |[1.18, 1.98] |1.58 |
abcde00000qMq00002|[clientes, contentos, servi]|[0.68, 2.1, 2.68] |1.8200000000000003|
abcde00000qMQ00001|[casa, alejado, buen, gusto]|[1.68, 2.8, 1.03, 3.68]|2.2975 |
+------------------+----------------------------+-----------------------+------------------+

In [21]:
from pyspark.sql import functions as F
df1.join(df2, F.expr("""array_contains(MeaningfulWords,word)"""))\
   .groupBy("ID").agg(F.first("MeaningfulWords").alias("MeaningfullWords")\
                      ,F.collect_list("score").alias("ScoreList"))\
   .withColumn("MeanScore", F.expr("""aggregate((transform(ScoreList,x->double(x)))\
                                      ,0,(x,acc)-> acc+x,acc->(acc/ size(Scorelist)))""")).show(truncate=False)

In [22]:
df1.show()

#+------------------+----------------------------+
#|ID                |MeaningfulWords             |
#+------------------+----------------------------+
#|abcde00000qMQ00001|[casa, alejado, buen, gusto]|
#|abcde00000qMq00002|[clientes, contentos, servi]|
#|abcde00000qMQ00003|[resto, bien]               |
#+------------------+----------------------------+

df2.show()

#+-----+---------+
#|score|     word|
#+-----+---------+
#| 1.68|     casa|
#|  2.8|  alejado|
#| 1.03|     buen|
#| 3.68|    gusto|
#| 0.68| clientes|
#|  2.1|contentos|
#| 2.68|    servi|
#| 1.18|    resto|
#| 1.98|     bien|
#+-----+---------+


from pyspark.sql import functions as F
df1.join(df2, F.expr("""array_contains(MeaningfulWords,word)"""))\
   .groupBy("ID").agg(F.first("MeaningfulWords").alias("MeaningfullWords")\
                      ,F.collect_list("score").alias("ScoreList")\
                      ,F.mean("score").alias("MeanScore"))\
                      .show(truncate=False)

#+------------------+----------------------------+-----------------------+------------------+
#|ID                |MeaningfullWords            |ScoreList              |MeanScore         |
#+------------------+----------------------------+-----------------------+------------------+
#|abcde00000qMQ00003|[resto, bien]               |[1.18, 1.98]           |1.58              |
#|abcde00000qMq00002|[clientes, contentos, servi]|[0.68, 2.1, 2.68]      |1.8200000000000003|
#|abcde00000qMQ00001|[casa, alejado, buen, gusto]|[1.68, 2.8, 1.03, 3.68]|2.2975            |
#+------------------+----------------------------+-----------------------+------------------+

+------------------+----------------------------+-----------------------+---------+
ID |MeaningfullWords |ScoreList |MeanScore|
+------------------+----------------------------+-----------------------+---------+
abcde00000qMQ00003|[resto, bien] |[1.18, 1.98] |3.16 |
abcde00000qMq00002|[clientes, contentos, servi]|[0.68, 2.1, 2.68] |5.46 |
abcde00000qMQ00001|[casa, alejado, buen, gusto]|[1.68, 2.8, 1.03, 3.68]|9.19 |
+------------------+----------------------------+-----------------------+---------+

In [23]:
from pyspark.sql import functions as F
df1.join(df2, F.expr("""array_contains(MeaningfulWords,word)"""))\
   .groupBy("ID").agg(F.first("MeaningfulWords").alias("MeaningfullWords")\
                      ,F.collect_list("score").alias("ScoreList"))\
   .withColumn("MeanScore", F.expr("""aggregate((transform(ScoreList,x->int(x*100)))\
                                      ,0,(x,acc)-> acc+x,acc->(acc/100)/ size(Scorelist))""")).show(truncate=False)


+------------------+----------------------------+-----------------------+---------+
ID |MeaningfullWords |ScoreList |MeanScore|
+------------------+----------------------------+-----------------------+---------+
abcde00000qMQ00003|[resto, bien] |[1.18, 1.98] |1.58 |
abcde00000qMq00002|[clientes, contentos, servi]|[0.68, 2.1, 2.68] |1.82 |
abcde00000qMQ00001|[casa, alejado, buen, gusto]|[1.68, 2.8, 1.03, 3.68]|2.2975 |
+------------------+----------------------------+-----------------------+---------+

In [24]:
list=[['A',          'yes'        ,1      ,1000],
      ['A',           'no'         ,2      ,100],
      ['A' ,          'no'         ,3      ,100],
      ['A'  ,         'no'         ,4      ,100],
      ['A'   ,        'no'         ,5      ,100],
      ['B'    ,       'yes'        ,1      ,2000],
      ['B'     ,     'no'         ,2      ,200],
      ['B'       ,    'no'         ,3      ,100],
      ['B'      ,     'no'         ,4      ,100],
      ['B'       ,    'no'         ,5      ,200],
      ['C'         ,  'yes'        ,1      ,3000],
      ['C'        ,   'no'         ,2      ,100],
      ['C'          , 'no'         ,3      ,100],
      ['C'           ,'no'         ,4      ,200],
      ['C'           ,'no'         ,5      ,200]]

df=spark.createDataFrame(list,['Group','Risk Group','State','Value'])

df.show()

+-----+----------+-----+-----+
Group|Risk Group|State|Value|
+-----+----------+-----+-----+
 A| yes| 1| 1000|
 A| no| 2| 100|
 A| no| 3| 100|
 A| no| 4| 100|
 A| no| 5| 100|
 B| yes| 1| 2000|
 B| no| 2| 200|
 B| no| 3| 100|
 B| no| 4| 100|
 B| no| 5| 200|
 C| yes| 1| 3000|
 C| no| 2| 100|
 C| no| 3| 100|
 C| no| 4| 200|
 C| no| 5| 200|
+-----+----------+-----+-----+

In [25]:
from pyspark.sql import functions as F
from pyspark.sql.window import Window

w=Window().partitionBy("Group")
df.withColumn("Risk Group Value", F.sum(F.when(F.col("Risk Group")=='yes',F.col("Value"))).over(w))\
  .filter(F.col("Risk Group")!='yes')\
  .orderBy("Group").show()

+-----+----------+-----+-----+----------------+
Group|Risk Group|State|Value|Risk Group Value|
+-----+----------+-----+-----+----------------+
 A| no| 2| 100| 1000|
 A| no| 3| 100| 1000|
 A| no| 4| 100| 1000|
 A| no| 5| 100| 1000|
 B| no| 2| 200| 2000|
 B| no| 3| 100| 2000|
 B| no| 4| 100| 2000|
 B| no| 5| 200| 2000|
 C| no| 2| 100| 3000|
 C| no| 5| 200| 3000|
 C| no| 3| 100| 3000|
 C| no| 4| 200| 3000|
+-----+----------+-----+-----+----------------+

In [26]:
Group - Risk Group - State - Value - Risk Group Value
A           no         2      100         1000
A           no         3      100         1000
A           no         4      100         1000
A           no         5      100         1000 
B           no         2      200         2000
B           no         3      100         2000
B           no         4      100         2000
B           no         5      200         2000
C           no         2      100         3000
C           no         3      100         3000
C           no         4      200         3000
C           no         5      200         3000

In [27]:
from pyspark.sql import functions as F

list=[[['a', 'b','b','c']],
      [['b', 'c', 'd']]]


df=spark.createDataFrame(list,['atr_list'])

df.show()

+------------+
 atr_list|
+------------+
[a, b, b, c]|
 [b, c, d]|
+------------+

In [28]:
df.show()

#+------------+
#|    atr_list|
#+------------+
#|[a, b, b, c]|
#|   [b, c, d]|
#+------------+

from pyspark.sql import functions as F

elements=['a','b','c','d']

collected=df.withColumn("a", F.when(F.array_contains("atr_list","a"),F.expr("""size(filter(atr_list,x->x='a'))""")).otherwise(F.lit(0)))\
  .withColumn("b", F.when(F.array_contains("atr_list","b"),F.expr("""size(filter(atr_list,x->x='b'))""")))\
  .withColumn("c", F.when(F.array_contains("atr_list","c"),F.expr("""size(filter(atr_list,x->x='c'))""")))\
  .withColumn("d", F.when(F.array_contains("atr_list","d"),F.expr("""size(filter(atr_list,x->x='d'))""")))\
  .agg(*[F.sum(x).alias(x) for x in elements])\
  .collect()[0]

{elements[i]: [x for x in collected][i] for i in range(len(elements))} 

Out[31]: {'a': 1, 'b': 3, 'c': 2, 'd': 1}

+------------+
 atr_list|
+------------+
[a, b, b, c]|
 [b, c, d]|
+------------+

Out[31]: {'a': 1, 'b': 3, 'c': 2, 'd': 1}

In [29]:
df.show()

#+------------+
#|    atr_list|
#+------------+
#|[a, b, b, c]|
#|   [b, c, d]|
#+------------+

elements=['a','b','c','d']
df.withColumn("struct", F.struct(*[(F.struct(F.expr("size(filter(atr_list,x->x={}))"\
                                                    .format("'"+y+"'"))).alias(y)) for y in elements]))\
  .select(*[F.sum(F.col("struct.{}.col1".format(x))).alias(x) for x in elements])\
  .collect()[0]

{elements[i]: [x for x in collected][i] for i in range(len(elements))} 
    
  

+------------+
 atr_list|
+------------+
[a, b, b, c]|
 [b, c, d]|
+------------+

Out[59]: {'a': 1, 'b': 3, 'c': 2, 'd': 1}

In [30]:
elements=['a','b','c','d']
df.withColumn("struct", F.struct(*[(F.struct(F.expr("size(filter(atr_list,x->x={}))"\
                                                    .format("'"+y+"'"))).alias(y)) for y in elements]))\
  .select(*[F.sum(F.col("struct.{}.col1".format(x))).alias(x) for x in elements])\
  .show()

+---+---+---+---+
 a| b| c| d|
+---+---+---+---+
 1| 3| 2| 1|
+---+---+---+---+

In [31]:
["size(filter(atr_list,x->x={}))".format("'"+y+"'") for y in ['a','b','c','d']]

Out[46]: ["size(filter(atr_list,x->x='a'))",
 "size(filter(atr_list,x->x='b'))",
 "size(filter(atr_list,x->x='c'))",
 "size(filter(atr_list,x->x='d'))"]

In [32]:
counter

Out[25]: {'a': 1, 'b': 3, 'c': 2, 'd': 1}

In [33]:
['a','b','c','d'][1]

Out[23]: 'b'

In [34]:
df.withColumn("atr_list",F.expr("""transform(array_distinct(atr_list), x-> struct(x,\
                                  size(filter(atr_list,y->y=x))))"""))\
   .show(truncate=False)

In [35]:
df.withColumn("atr_list",F.expr("""transform(array_distinct(atr_list), x-> map(x,\
                                  size(filter(atr_list,y->y=x))))"""))\
   .show(truncate=False)

root
-- atr_list: array (nullable = true)
 |-- element: map (containsNull = false)
 | |-- key: string
 | |-- value: integer (valueContainsNull = false)

In [36]:
df.agg(F.flatten(F.collect_list("atr_list")).alias("atr_list"))\
   .withColumn("atr_list",F.expr("""transform(array_distinct(atr_list), x-> map(x,\
                                  size(filter(atr_list,y->y=x))))""")).collect()[0][0]


Out[27]: [{'a': 1}, {'b': 3}, {'c': 2}, {'d': 1}]

In [37]:
df.withColumn("atr_list", F.explode("atr_list"))\
  .groupBy("atr_list").count().show()

+--------+-----+
atr_list|count|
+--------+-----+
 d| 1|
 c| 2|
 b| 3|
 a| 1|
+--------+-----+

In [38]:
df.withColumn("atr_list", F.expr("""transform(array_distinct(atr_list), x-> struct(x as item,\
                                  size(filter(atr_list,y->y=x)) as sum)))"""))\
  .select("atr_list.item","atr_list.sum")\
  .groupBy("item").agg(F.sum("sum").alias("SUM"))\
  .show()

In [39]:
df.withColumn("atr_list2", F.expr("""transform(atr_list, x-> map(x,\
                               (aggregate(atr_list,0,(y,acc)->IF(x=y,int(acc)+int(1),int(acc))\
                               ,acc->acc))))"""))\
  .show(truncate=False)

In [40]:
df.withColumn("atr_list2", F.expr("""aggregate()"""))

In [41]:
df1 = spark.createDataFrame(
     [
     ('ll',5),
     ('yy',6),
     ],
     ('x','days')
    )

df = spark.createDataFrame(
[
    ('ll','2020-01-05','1','10'),
    ('ll','2020-01-06','1','10'),
    ('ll','2020-01-07','1','10'),
    ('ll','2020-01-08','1','10'),
    ('ll','2020-01-09','1','10'),
    ('ll','2020-01-10','1','10'),
    ('ll','2020-01-11','1','10'),
    ('ll','2020-01-12','1','10'),
    ('ll','2020-01-05','2','30'),
    ('ll','2020-01-06','2','30'),
    ('ll','2020-01-07','2','30'),
    ('ll','2020-01-08','2','30'),
    ('ll','2020-01-09','2','30'),
    ('ll','2020-01-10','2','10'),
    ('ll','2020-01-11','2','10'),
    ('ll','2020-01-12','2','10'),
    ('yy','2020-01-05','1','20'),
    ('yy','2020-01-06','1','20'),
    ('yy','2020-01-07','1','20'),
    ('yy','2020-01-08','1','20'),
    ('yy','2020-01-09','1','20'),
    ('yy','2020-01-10','1','20'),
    ('yy','2020-01-11','1','20'),
    ('yy','2020-01-12','1','20'),
    ('yy','2020-01-05','2','40'),
    ('yy','2020-01-06','2','40'),
    ('yy','2020-01-07','2','40'),
    ('yy','2020-01-08','2','40'),
    ('yy','2020-01-09','2','40'),
    ('yy','2020-01-10','2','40'),
    ('yy','2020-01-11','2','40'),
    ('yy','2020-01-12','2','40')
     ],
    ('x','date','flag','value')
    )
df1.show()
df.show()

+---+----+
 x|days|
+---+----+
 ll| 5|
 yy| 6|
+---+----+

+---+----------+----+-----+
 x| date|flag|value|
+---+----------+----+-----+
 ll|2020-01-05| 1| 10|
 ll|2020-01-06| 1| 10|
 ll|2020-01-07| 1| 10|
 ll|2020-01-08| 1| 10|
 ll|2020-01-09| 1| 10|
 ll|2020-01-10| 1| 10|
 ll|2020-01-11| 1| 10|
 ll|2020-01-12| 1| 10|
 ll|2020-01-05| 2| 30|
 ll|2020-01-06| 2| 30|
 ll|2020-01-07| 2| 30|
 ll|2020-01-08| 2| 30|
 ll|2020-01-09| 2| 30|
 ll|2020-01-10| 2| 10|
 ll|2020-01-11| 2| 10|
 ll|2020-01-12| 2| 10|
 yy|2020-01-05| 1| 20|
 yy|2020-01-06| 1| 20|
 yy|2020-01-07| 1| 20|
 yy|2020-01-08| 1| 20|
+---+----------+----+-----+
only showing top 20 rows

In [42]:
from pyspark.sql import functions as F
from pyspark.sql.window import Window

w=Window().partitionBy("x","flag").orderBy(F.to_date("date","yyyy-dd-MM"))
w1=Window().partitionBy("x","flag")
df.join(df1, ['x'])\
  .withColumn("rowNum", F.row_number().over(w))\
  .withColumn("expected_result", F.sum(F.when(F.col("rowNum")>F.col("days")\
                                     ,F.lit(None)).otherwise(F.col("value")))\
                                      .over(w1)).drop("days","rowNum").show()

+---+----------+----+-----+---------------+
 x| date|flag|value|expected_result|
+---+----------+----+-----+---------------+
 ll|2020-01-05| 1| 10| 50.0|
 ll|2020-01-06| 1| 10| 50.0|
 ll|2020-01-07| 1| 10| 50.0|
 ll|2020-01-08| 1| 10| 50.0|
 ll|2020-01-09| 1| 10| 50.0|
 ll|2020-01-10| 1| 10| 50.0|
 ll|2020-01-11| 1| 10| 50.0|
 ll|2020-01-12| 1| 10| 50.0|
 ll|2020-01-05| 2| 30| 150.0|
 ll|2020-01-06| 2| 30| 150.0|
 ll|2020-01-07| 2| 30| 150.0|
 ll|2020-01-08| 2| 30| 150.0|
 ll|2020-01-09| 2| 30| 150.0|
 ll|2020-01-10| 2| 10| 150.0|
 ll|2020-01-11| 2| 10| 150.0|
 ll|2020-01-12| 2| 10| 150.0|
 yy|2020-01-05| 1| 20| 120.0|
 yy|2020-01-06| 1| 20| 120.0|
 yy|2020-01-07| 1| 20| 120.0|
 yy|2020-01-08| 1| 20| 120.0|
+---+----------+----+-----+---------------+
only showing top 20 rows

In [43]:
+---+----------+----+-----+---------------+
    |  x|      date|flag|value|expected_result|
    +---+----------+----+-----+---------------+
    | ll|2020-01-05|   1|   10|             50|
    | ll|2020-01-06|   1|   10|             50|
    | ll|2020-01-07|   1|   10|             50|
    | ll|2020-01-08|   1|   10|             50|
    | ll|2020-01-09|   1|   10|             50|
    | ll|2020-01-10|   1|   10|             50|
    | ll|2020-01-11|   1|   10|             50|
    | ll|2020-01-12|   1|   10|             50|
    | ll|2020-01-05|   2|   30|            150|
    | ll|2020-01-06|   2|   30|            150|
    | ll|2020-01-07|   2|   30|            150|
    | ll|2020-01-08|   2|   30|            150|
    | ll|2020-01-09|   2|   30|            150|
    | ll|2020-01-10|   2|   10|            150|
    | ll|2020-01-11|   2|   10|            150|
    | ll|2020-01-12|   2|   10|            150|
    | yy|2020-01-05|   1|   20|            120|
    | yy|2020-01-06|   1|   20|            120|
    | yy|2020-01-07|   1|   20|            120|
    | yy|2020-01-08|   1|   20|            120|
    | yy|2020-01-09|   1|   20|            120|
    | yy|2020-01-10|   1|   20|            120|
    | yy|2020-01-11|   1|   20|            120|
    | yy|2020-01-12|   1|   20|            120|
    | yy|2020-01-05|   2|   40|            240|
    | yy|2020-01-06|   2|   40|            240|
    | yy|2020-01-07|   2|   40|            240|
    | yy|2020-01-08|   2|   40|            240|
    | yy|2020-01-09|   2|   40|            240|
    | yy|2020-01-10|   2|   40|            240|
    | yy|2020-01-11|   2|   40|            240|
    | yy|2020-01-12|   2|   40|            240|
    +---+----------+----+-----+---------------+

In [44]:
list=[[1, 1, '12/3/2020'],
     [1,  0, '12/4/2020'],
     [1,   1, '12/5/2020'],
     [1,   1, '12/6/2020'],
     [1,   0, '12/7/2020'],
     [1,   1, '12/8/2020'],
          [1,   0, '12/9/2020'],
          [1,   0,'12/10/2020'],
          [1,   0,'12/11/2020'],
          [1,   1,'12/12/2020'],
          [2,   1, '12/1/2020'],
          [2,   0, '12/2/2020'],
          [2,   0, '12/3/2020'],
         [2,  1, '12/4/2020']]


df=spark.createDataFrame(list,['customer_id','Flag','trx_date'])
df.show()

+-----------+----+----------+
customer_id|Flag| trx_date|
+-----------+----+----------+
 1| 1| 12/3/2020|
 1| 0| 12/4/2020|
 1| 1| 12/5/2020|
 1| 1| 12/6/2020|
 1| 0| 12/7/2020|
 1| 1| 12/8/2020|
 1| 0| 12/9/2020|
 1| 0|12/10/2020|
 1| 0|12/11/2020|
 1| 1|12/12/2020|
 2| 1| 12/1/2020|
 2| 0| 12/2/2020|
 2| 0| 12/3/2020|
 2| 1| 12/4/2020|
+-----------+----+----------+

In [45]:
from pyspark.sql import functions as F
from pyspark.sql.window import Window

w=Window().orderBy("customer_id","trx_date")
w1=Window().partitionBy("Flag2").orderBy("trx_date").rowsBetween(Window.unboundedPreceding,Window.unboundedFollowing)
df.withColumn("trx_date", F.to_date("trx_date", "MM/dd/yyyy"))\
  .withColumn("Flag2", F.sum("Flag").over(w))\
  .withColumn("destination", F.when(F.col("Flag")==0, F.first("trx_date").over(w1)).otherwise(F.col("trx_date")))\
  .withColumn("trx_date", F.date_format("trx_date","MM/dd/yyyy"))\
  .withColumn("destination", F.date_format("destination", "MM/dd/yyyy"))\
  .orderBy("customer_id","trx_date").drop("Flag2").show()

+-----------+----+----------+-----------+
customer_id|Flag| trx_date|destination|
+-----------+----+----------+-----------+
 1| 1|12/03/2020| 12/03/2020|
 1| 0|12/04/2020| 12/03/2020|
 1| 1|12/05/2020| 12/05/2020|
 1| 1|12/06/2020| 12/06/2020|
 1| 0|12/07/2020| 12/06/2020|
 1| 1|12/08/2020| 12/08/2020|
 1| 0|12/09/2020| 12/08/2020|
 1| 0|12/10/2020| 12/08/2020|
 1| 0|12/11/2020| 12/08/2020|
 1| 1|12/12/2020| 12/12/2020|
 2| 1|12/01/2020| 12/01/2020|
 2| 0|12/02/2020| 12/01/2020|
 2| 0|12/03/2020| 12/01/2020|
 2| 1|12/04/2020| 12/04/2020|
+-----------+----+----------+-----------+

In [46]:
+-----------+----+----------+-----------+
|customer_id|Flag|  trx_date|destination|
+-----------+----+----------+-----------+
|          1|   1| 12/3/2020|  12/3/2020|
|          1|   0| 12/4/2020|  12/3/2020|
|          1|   1| 12/5/2020|  12/5/2020|
|          1|   1| 12/6/2020|  12/6/2020|
|          1|   0| 12/7/2020|  12/6/2020|
|          1|   1| 12/8/2020|  12/8/2020|
|          1|   0| 12/9/2020|  12/8/2020|
|          1|   0|12/10/2020|  12/8/2020|
|          1|   0|12/11/2020|  12/8/2020|
|          1|   1|12/12/2020| 12/12/2020|
|          2|   1| 12/1/2020|  12/1/2020|
|          2|   0| 12/2/2020|  12/1/2020|
|          2|   0| 12/3/2020|  12/1/2020|
|          2|   1| 12/4/2020|  12/4/2020|
+-----------+----+----------+-----------+

In [47]:
list=[['2020-01-13T22:22:10.000+0000',1,173736,3043,2996],
     ['2020-01-13T22:43:19.000+0000',1,173775,3042,2996],
     ['2020-01-14T22:43:19.000+0000',1,173775,3042,2996],
     ['2020-01-15T22:43:19.000+0000',1,173775,3042,2996],
     ['2020-01-13T22:22:10.000+0000',2,257624,1500,53],
     ['2020-01-13T22:43:19.000+0000',2,257625,1500,65],
     ['2020-01-13T22:22:10.000+0000',1,173736,3043,2996],
     ['2020-01-13T22:43:19.000+0000',1,173775,3042,2996],
     ['2020-01-14T22:43:19.000+0000',1,173775,3042,2996],
     ['2020-01-15T22:43:19.000+0000',1,173775,3042,2996],
     ['2020-01-13T22:22:10.000+0000',2,257624,1500,53],
     ['2020-01-13T22:43:19.000+0000',2,257625,1500,65],
     ['2020-01-13T22:22:10.000+0000',1,173736,3043,2996],
     ['2020-01-13T22:43:19.000+0000',1,173775,3042,2996],
     ['2020-01-14T22:43:19.000+0000',1,173775,3042,2996],
     ['2020-01-15T22:43:19.000+0000',1,173775,3042,2996],
     ['2020-01-13T22:22:10.000+0000',2,257624,1500,53],
     ['2020-01-13T22:43:19.000+0000',2,257625,1500,65],
     ['2020-01-13T22:22:10.000+0000',1,173736,3043,2996],
     ['2020-01-13T22:43:19.000+0000',1,173775,3042,2996],
     ['2020-01-14T22:43:19.000+0000',1,173775,3042,2996],
     ['2020-01-15T22:43:19.000+0000',1,173775,3042,2996],
     ['2020-01-13T22:22:10.000+0000',2,257624,1500,53],
     ['2020-01-13T22:43:19.000+0000',2,257625,1500,65]]

df=spark.createDataFrame(list,['date','id','stat1','stat2','stat3'])

df.show(truncate=False)

+----------------------------+---+------+-----+-----+
date |id |stat1 |stat2|stat3|
+----------------------------+---+------+-----+-----+
2020-01-13T22:22:10.000+0000|1 |173736|3043 |2996 |
2020-01-13T22:43:19.000+0000|1 |173775|3042 |2996 |
2020-01-14T22:43:19.000+0000|1 |173775|3042 |2996 |
2020-01-15T22:43:19.000+0000|1 |173775|3042 |2996 |
2020-01-13T22:22:10.000+0000|2 |257624|1500 |53 |
2020-01-13T22:43:19.000+0000|2 |257625|1500 |65 |
2020-01-13T22:22:10.000+0000|1 |173736|3043 |2996 |
2020-01-13T22:43:19.000+0000|1 |173775|3042 |2996 |
2020-01-14T22:43:19.000+0000|1 |173775|3042 |2996 |
2020-01-15T22:43:19.000+0000|1 |173775|3042 |2996 |
2020-01-13T22:22:10.000+0000|2 |257624|1500 |53 |
2020-01-13T22:43:19.000+0000|2 |257625|1500 |65 |
2020-01-13T22:22:10.000+0000|1 |173736|3043 |2996 |
2020-01-13T22:43:19.000+0000|1 |173775|3042 |2996 |
2020-01-14T22:43:19.000+0000|1 |173775|3042 |2996 |
2020-01-15T22:43:19.000+0000|1 |173775|3042 |2996 |
2020-01-13T22:22:10.000+0000|2 |257624|1500 |53 |
2020-01-13T22:43:19.000+0000|2 |257625|1500 |65 |
2020-01-13T22:22:10.000+0000|1 |173736|3043 |2996 |
2020-01-13T22:43:19.000+0000|1 |173775|3042 |2996 |
+----------------------------+---+------+-----+-----+
only showing top 20 rows

In [48]:
df1,df2,df3=df.randomSplit([0.3,0.3,0.4])

print(df1.count(),df2.count(),df3.count())

6 10 8

In [49]:
def split(df):
    if df.count()>2:
        df1,df2,df3=df.randomSplit([0.2,0.2,0.2],24)
        return df1.show(),df2.show(),df3.show()
    else:
        return df.show()
      
split(df)


+--------------------+---+------+-----+-----+
 date| id| stat1|stat2|stat3|
+--------------------+---+------+-----+-----+
2020-01-13T22:22:...| 1|173736| 3043| 2996|
2020-01-13T22:22:...| 2|257624| 1500| 53|
2020-01-13T22:43:...| 2|257625| 1500| 65|
+--------------------+---+------+-----+-----+

+----+---+-----+-----+-----+
date| id|stat1|stat2|stat3|
+----+---+-----+-----+-----+
+----+---+-----+-----+-----+

+--------------------+---+------+-----+-----+
 date| id| stat1|stat2|stat3|
+--------------------+---+------+-----+-----+
2020-01-13T22:43:...| 1|173775| 3042| 2996|
2020-01-14T22:43:...| 1|173775| 3042| 2996|
2020-01-15T22:43:...| 1|173775| 3042| 2996|
+--------------------+---+------+-----+-----+

Out[3]: (None, None, None)

--------------------------------------------------------------------------- 
 NameError Traceback (most recent call last)
 <command-3071577957973779> in <module> 
 1 df1 . show ( ) 
 2 df2 . show ( ) 
 ----> 3 df3 . show ( ) 

 NameError : name 'df3' is not defined

In [51]:
from pyspark.sql import functions as F
from pyspark.sql.window import Window

w=Window().partitionBy("id","date1").orderBy("date")
w2=Window().partitionBy("id","date1")
df.withColumn("date", F.to_timestamp("date","yyyy-MM-dd'T'HH:mm:ss"))\
  .withColumn("date1", F.to_date("date"))\
  .withColumn("rownum", F.row_number().over(w))\
  .withColumn("max", F.max("rownum").over(w2))\
  .filter('rownum=max').drop("date1","rownum","max")\
  .orderBy("id","date").show(truncate=False)

+-------------------+---+------+-----+-----+
date |id |stat1 |stat2|stat3|
+-------------------+---+------+-----+-----+
2020-01-13 22:43:19|1 |173775|3042 |2996 |
2020-01-14 22:43:19|1 |173775|3042 |2996 |
2020-01-15 22:43:19|1 |173775|3042 |2996 |
2020-01-13 22:43:19|2 |257625|1500 |65 |
+-------------------+---+------+-----+-----+

In [52]:
from pyspark.sql import functions as F
from pyspark.sql.window import Window

w=Window().partitionBy("id","date1").orderBy("date").rangeBetween(Window.unboundedPreceding,Window.unboundedFollowing)
df.withColumn("date", F.to_timestamp("date","yyyy-MM-dd'T'HH:mm:ss"))\
  .withColumn("date1", F.to_date("date"))\
  .withColumn("rownum", F.last("date").over(w))\
  .filter('rownum=date').drop("date1","rownum").orderBy("id","date").show(truncate=False)

+-------------------+---+------+-----+-----+
date |id |stat1 |stat2|stat3|
+-------------------+---+------+-----+-----+
2020-01-13 22:43:19|1 |173775|3042 |2996 |
2020-01-14 22:43:19|1 |173775|3042 |2996 |
2020-01-15 22:43:19|1 |173775|3042 |2996 |
2020-01-13 22:43:19|2 |257625|1500 |65 |
+-------------------+---+------+-----+-----+

In [53]:
df = spark.createDataFrame([['x',1, "9999"], ['x',2, "120"], ['x',3, "102"], ['x',4, "3000"],['x',5, "299"],['x',6, "100"]],['id',"row_number", "time_diff"])

df.show()

+---+----------+---------+
 id|row_number|time_diff|
+---+----------+---------+
 x| 1| 9999|
 x| 2| 120|
 x| 3| 102|
 x| 4| 3000|
 x| 5| 299|
 x| 6| 100|
+---+----------+---------+

In [54]:
w=Window().partitionBy("id").orderBy("row_number")
df.withColumn("new_row_number", F.when(F.col("time_diff")>=160,F.col("row_number")))\
  .withColumn("new_row_number", F.last("new_row_number",True).over(w)).show()

+---+----------+---------+--------------+
 id|row_number|time_diff|new_row_number|
+---+----------+---------+--------------+
 x| 1| 9999| 1|
 x| 2| 120| 1|
 x| 3| 102| 1|
 x| 4| 3000| 4|
 x| 5| 299| 5|
 x| 6| 100| 5|
+---+----------+---------+--------------+

In [55]:
list=[[ 5,     'a'],
      [  2,     'b'],
      [  3,     'c'],
      [ 6,    'a'],
      [ 7,     'b']]

dfviol=spark.createDataFrame(list,['Number','Letter'])


list1=[[ 1,     'a',                1,                 5,                 6,                10],
       [ 2,     'a',                7,                9,                 0,                  4],
       [ 3,     'a',               11,                15,                10,                 14],
       [ 4,     'b',                1,                 5,                 0,                  4],
       [ 5,     'b',                7,                 9,                 6,                 10],
       [ 6,     'c',                1,                 5,                 0,                  4],
       [ 7,     'c',                7,                 9,                 6,                 10],
       [ 8,     'c',               11,                15,                10,                 14]]


dfcent=spark.createDataFrame(list1,['ID','Letter','Num_range_low_odd','Num_range_high_odd','Num_range_low_even','Num_range_high_even'])

dfviol.show()
dfcent.show()

+------+------+
Number|Letter|
+------+------+
 5| a|
 2| b|
 3| c|
 6| a|
 7| b|
+------+------+

+---+------+-----------------+------------------+------------------+-------------------+
 ID|Letter|Num_range_low_odd|Num_range_high_odd|Num_range_low_even|Num_range_high_even|
+---+------+-----------------+------------------+------------------+-------------------+
 1| a| 1| 5| 6| 10|
 2| a| 7| 9| 0| 4|
 3| a| 11| 15| 10| 14|
 4| b| 1| 5| 0| 4|
 5| b| 7| 9| 6| 10|
 6| c| 1| 5| 0| 4|
 7| c| 7| 9| 6| 10|
 8| c| 11| 15| 10| 14|
+---+------+-----------------+------------------+------------------+-------------------+

In [56]:
joinCondition = F.when(dfviol.Number%2== 0, [(dfcent.Num_range_low_even <= dfviol.Number) & (dfcent.Num_range_high_even >= dfviol.Number)]).otherwise([(dfcent.Num_range_low_odd <= dfviol.Number) & (dfcent.Num_range_high_odd >= dfviol.Number)])

df_full = dfviol.join(dfcent,[dfviol.Letter == dfcent.Letter, joinCondition], how='inner')
df_full.show()

--------------------------------------------------------------------------- 
 TypeError Traceback (most recent call last)
 <command-2734917699186196> in <module> 
 ----> 1 joinCondition = F . when ( dfviol . Number % 2 == 0 , [ ( dfcent . Num_range_low_even <= dfviol . Number ) & ( dfcent . Num_range_high_even >= dfviol . Number ) ] ) . otherwise ( [ ( dfcent . Num_range_low_odd <= dfviol . Number ) & ( dfcent . Num_range_high_odd >= dfviol . Number ) ] ) 
 2 
 3 df_full = dfviol . join ( dfcent , [ dfviol . Letter == dfcent . Letter , joinCondition ] , how = 'inner' ) 
 4 df_full . show ( ) 

 /databricks/spark/python/pyspark/sql/functions.py in when (condition, value) 
 745 raise TypeError ( "condition should be a Column" ) 
 746 v = value . _jc if isinstance ( value , Column ) else value
 --> 747 jc = sc . _jvm . functions . when ( condition . _jc , v ) 
 748 return Column ( jc ) 
 749 

 /databricks/spark/python/lib/py4j-0.10.7-src.zip/py4j/java_gateway.py in __call__ (self, *args) 
 1246 
 1247 def __call__ ( self , * args ) : 
 -> 1248 args_command , temp_args = self . _build_args ( * args ) 
 1249 
 1250 command = proto . CALL_COMMAND_NAME + \ 

 /databricks/spark/python/lib/py4j-0.10.7-src.zip/py4j/java_gateway.py in _build_args (self, *args) 
 1210 def _build_args ( self , * args ) : 
 1211 if self . converters is not None and len ( self . converters ) > 0 : 
 -> 1212 ( new_args , temp_args ) = self . _get_args ( args ) 
 1213 else : 
 1214 new_args = args

 /databricks/spark/python/lib/py4j-0.10.7-src.zip/py4j/java_gateway.py in _get_args (self, args) 
 1197 for converter in self . gateway_client . converters : 
 1198 if converter . can_convert ( arg ) : 
 -> 1199 temp_arg = converter . convert ( arg , self . gateway_client ) 
 1200 temp_args . append ( temp_arg ) 
 1201 new_args . append ( temp_arg ) 

 /databricks/spark/python/lib/py4j-0.10.7-src.zip/py4j/java_collections.py in convert (self, object, gateway_client) 
 499 java_list = ArrayList ( ) 
 500 for element in object : 
 --> 501 java_list . add ( element ) 
 502 return java_list
 503 

 /databricks/spark/python/lib/py4j-0.10.7-src.zip/py4j/java_gateway.py in __call__ (self, *args) 
 1246 
 1247 def __call__ ( self , * args ) : 
 -> 1248 args_command , temp_args = self . _build_args ( * args ) 
 1249 
 1250 command = proto . CALL_COMMAND_NAME + \ 

 /databricks/spark/python/lib/py4j-0.10.7-src.zip/py4j/java_gateway.py in _build_args (self, *args) 
 1210 def _build_args ( self , * args ) : 
 1211 if self . converters is not None and len ( self . converters ) > 0 : 
 -> 1212 ( new_args , temp_args ) = self . _get_args ( args ) 
 1213 else : 
 1214 new_args = args

 /databricks/spark/python/lib/py4j-0.10.7-src.zip/py4j/java_gateway.py in _get_args (self, args) 
 1197 for converter in self . gateway_client . converters : 
 1198 if converter . can_convert ( arg ) : 
 -> 1199 temp_arg = converter . convert ( arg , self . gateway_client ) 
 1200 temp_args . append ( temp_arg ) 
 1201 new_args . append ( temp_arg ) 

 /databricks/spark/python/lib/py4j-0.10.7-src.zip/py4j/java_collections.py in convert (self, object, gateway_client) 
 498 ArrayList = JavaClass ( "java.util.ArrayList" , gateway_client ) 
 499 java_list = ArrayList ( ) 
 --> 500 for element in object : 
 501 java_list . add ( element ) 
 502 return java_list

 /databricks/spark/python/pyspark/sql/column.py in __iter__ (self) 
 342 
 343 def __iter__ ( self ) : 
 --> 344 raise TypeError ( "Column is not iterable" ) 
 345 
 346 # string methods 

 TypeError : Column is not iterable

In [57]:
#joinCondition = when(dfviol.Number%2== 0, [dfcent.Num_range_low_even <= dfviol.Number,dfcent.Num_range_high_even >= #dfviol.Number]).otherwise([dfcent.Num_range_low_odd <= dfviol.Number,dfcent.Num_range_high_odd >= dfviol.Number])

df1 = dfviol.filter(F.col("Number")%2==0).join(dfcent,[dfviol.Letter == dfcent.Letter, dfcent.Num_range_low_even <= dfviol.Number,dfcent.Num_range_high_even >= dfviol.Number], how='inner')
df2 = dfviol.filter(F.col("Number")%2!=0).join(dfcent,[dfviol.Letter == dfcent.Letter,dfcent.Num_range_low_odd <= dfviol.Number,dfcent.Num_range_high_odd >= dfviol.Number], how='inner')

df1.union(df2).show()

+------+------+---+------+-----------------+------------------+------------------+-------------------+
Number|Letter| ID|Letter|Num_range_low_odd|Num_range_high_odd|Num_range_low_even|Num_range_high_even|
+------+------+---+------+-----------------+------------------+------------------+-------------------+
 2| b| 4| b| 1| 5| 0| 4|
 6| a| 1| a| 1| 5| 6| 10|
 3| c| 6| c| 1| 5| 0| 4|
 7| b| 5| b| 7| 9| 6| 10|
 5| a| 1| a| 1| 5| 6| 10|
+------+------+---+------+-----------------+------------------+------------------+-------------------+

In [58]:
#joinCondition = when(dfviol.Number%2== 0, [dfcent.Num_range_low_even <= dfviol.Number,dfcent.Num_range_high_even >= #dfviol.Number]).otherwise([dfcent.Num_range_low_odd <= dfviol.Number,dfcent.Num_range_high_odd >= dfviol.Number])

df1 = dfviol.filter(F.col("Number")%2==0).join(dfcent,[dfviol.Letter == dfcent.Letter, dfcent.Num_range_low_even <= dfviol.Number,dfcent.Num_range_high_even >= dfviol.Number], how='inner')
df2 = dfviol.filter(F.col("Number")%2!=0).join(dfcent,[dfviol.Letter == dfcent.Letter,dfcent.Num_range_low_odd <= dfviol.Number,dfcent.Num_range_high_odd >= dfviol.Number], how='inner')

df1.union(df2).show()

In [59]:
joinCondition = F.expr("""IF(Number % 2==0, array((Num_range_low_even <= Number),(Num_range_high_even >= Number),(Letter)),array(((Num_range_low_odd <= Number) or (Num_range_high_odd >= Number) or (Letter1 == Letter))))""")

df_full = dfviol.withColumnRenamed("Letter","Letter1").join(dfcent,joinCondition, how='inner')
df_full.show()

--------------------------------------------------------------------------- 
 Py4JJavaError Traceback (most recent call last)
 /databricks/spark/python/pyspark/sql/utils.py in deco (*a, **kw) 
 62 try : 
 ---> 63 return f ( * a , ** kw ) 
 64 except py4j . protocol . Py4JJavaError as e : 

 /databricks/spark/python/lib/py4j-0.10.7-src.zip/py4j/protocol.py in get_return_value (answer, gateway_client, target_id, name) 
 327 "An error occurred while calling {0}{1}{2}.\n" . 
 --> 328 format(target_id, ".", name), value)
 329 else : 

 Py4JJavaError : An error occurred while calling o1822.join.
: org.apache.spark.sql.AnalysisException: cannot resolve 'array((`Num_range_low_even` <= `Number`), (`Num_range_high_even` >= `Number`), `Letter`)' due to data type mismatch: input to function array should all be the same type, but it's [boolean, boolean, string]; line 1 pos 18;
'Join Inner, 'IF(((Number#775L % cast(2 as bigint)) = cast(0 as bigint)), array((Num_range_low_even#783L <= Number#775L), (Num_range_high_even#784L >= Number#775L), Letter#780), array((((Num_range_low_odd#781L <= Number#775L) || (Num_range_high_odd#782L >= Number#775L)) || (Letter1#829 = Letter#780))))
:- Project [Number#775L, Letter#776 AS Letter1#829]
: +- LogicalRDD [Number#775L, Letter#776], false
+- LogicalRDD [ID#779L, Letter#780, Num_range_low_odd#781L, Num_range_high_odd#782L, Num_range_low_even#783L, Num_range_high_even#784L], false

	at org.apache.spark.sql.catalyst.analysis.package$AnalysisErrorAt.failAnalysis(package.scala:42)
	at org.apache.spark.sql.catalyst.analysis.CheckAnalysis$$anonfun$checkAnalysis$1$$anonfun$apply$3.applyOrElse(CheckAnalysis.scala:126)
	at org.apache.spark.sql.catalyst.analysis.CheckAnalysis$$anonfun$checkAnalysis$1$$anonfun$apply$3.applyOrElse(CheckAnalysis.scala:111)
	at org.apache.spark.sql.catalyst.trees.TreeNode$$anonfun$6.apply(TreeNode.scala:303)
	at org.apache.spark.sql.catalyst.trees.TreeNode$$anonfun$6.apply(TreeNode.scala:303)
	at org.apache.spark.sql.catalyst.trees.CurrentOrigin$.withOrigin(TreeNode.scala:76)
	at org.apache.spark.sql.catalyst.trees.TreeNode.transformUp(TreeNode.scala:302)
	at org.apache.spark.sql.catalyst.trees.TreeNode$$anonfun$5.apply(TreeNode.scala:300)
	at org.apache.spark.sql.catalyst.trees.TreeNode$$anonfun$5.apply(TreeNode.scala:300)
	at org.apache.spark.sql.catalyst.trees.TreeNode.org$apache$spark$sql$catalyst$trees$TreeNode$$mapChild$2(TreeNode.scala:322)
	at org.apache.spark.sql.catalyst.trees.TreeNode$$anonfun$8$$anonfun$apply$13.apply(TreeNode.scala:381)
	at scala.collection.TraversableLike$$anonfun$map$1.apply(TraversableLike.scala:234)
	at scala.collection.TraversableLike$$anonfun$map$1.apply(TraversableLike.scala:234)
	at scala.collection.mutable.ResizableArray$class.foreach(ResizableArray.scala:59)
	at scala.collection.mutable.ArrayBuffer.foreach(ArrayBuffer.scala:48)
	at scala.collection.TraversableLike$class.map(TraversableLike.scala:234)
	at scala.collection.AbstractTraversable.map(Traversable.scala:104)
	at org.apache.spark.sql.catalyst.trees.TreeNode$$anonfun$8.apply(TreeNode.scala:381)
	at org.apache.spark.sql.catalyst.trees.TreeNode.mapProductIterator(TreeNode.scala:207)
	at org.apache.spark.sql.catalyst.trees.TreeNode.mapChildren(TreeNode.scala:351)
	at org.apache.spark.sql.catalyst.trees.TreeNode.transformUp(TreeNode.scala:300)
	at org.apache.spark.sql.catalyst.plans.QueryPlan$$anonfun$transformExpressionsUp$1.apply(QueryPlan.scala:94)
	at org.apache.spark.sql.catalyst.plans.QueryPlan$$anonfun$transformExpressionsUp$1.apply(QueryPlan.scala:94)
	at org.apache.spark.sql.catalyst.plans.QueryPlan$$anonfun$3.apply(QueryPlan.scala:106)
	at org.apache.spark.sql.catalyst.plans.QueryPlan$$anonfun$3.apply(QueryPlan.scala:106)
	at org.apache.spark.sql.catalyst.trees.CurrentOrigin$.withOrigin(TreeNode.scala:76)
	at org.apache.spark.sql.catalyst.plans.QueryPlan.transformExpression$1(QueryPlan.scala:105)
	at org.apache.spark.sql.catalyst.plans.QueryPlan.org$apache$spark$sql$catalyst$plans$Qu

In [60]:
from pyspark.sql.functions import when

joinCondition = when(dfviol.Number%2== 0, (dfcent.Num_range_low_even <= dfviol.Number)&(dfcent.Num_range_high_even >= dfviol.Number)).otherwise((dfcent.Num_range_low_odd <= dfviol.Number) & (dfcent.Num_range_high_odd >= dfviol.Number))

df_full = dfviol.join(dfcent,[dfviol.Letter == dfcent.Letter, joinCondition], how='inner')
df_full.show()

+------+------+---+------+-----------------+------------------+------------------+-------------------+
Number|Letter| ID|Letter|Num_range_low_odd|Num_range_high_odd|Num_range_low_even|Num_range_high_even|
+------+------+---+------+-----------------+------------------+------------------+-------------------+
 3| c| 6| c| 1| 5| 0| 4|
 2| b| 4| b| 1| 5| 0| 4|
 7| b| 5| b| 7| 9| 6| 10|
 5| a| 1| a| 1| 5| 6| 10|
 6| a| 1| a| 1| 5| 6| 10|
+------+------+---+------+-----------------+------------------+------------------+-------------------+

In [61]:
https://medium.com/@murtazahash